# Quantitative Trading Strategies Using Python
## Technical Analysis, Statistical Testing,and Machine Learning
—
### Peng Liu

#######################################################################################################################

## CHAPTER 6
## Momentum Trading Strategy

##### Fetching relevant information from the web page

In [ ]:
!pip install botorch
!pip install -U statsmodels
!pip install ta
!pip install yfinance
from botorch.acquisition import ExpectedImprovement
from botorch.acquisition import qExpectedImprovement
from botorch.acquisition import UpperConfidenceBound
from botorch.acquisition.knowledge_gradient import qKnowledgeGradient
from botorch.fit import fit_gpytorch_mll
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import combinations
import math
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random
import requests
from statsmodels.tsa.stattools import adfuller
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import ta
import torch
import torch.nn as nn
import yfinance as yf

In [ ]:
def fetch_info():
    try:
        url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64;rv:101.0) Gecko/20100101 Firefox/101.0',
            'Accept': 'application/json',
            'Accept-Language': 'en-US,en;q=0.5',
        }
        #  Send GET request
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        #  Get the symbols table
        tables = soup.find_all('table')
        #  #  Convert table to dataframe
        df = pd.read_html(str(tables))[1]
        #  Cleanup
        df.drop(columns=['Notes'], inplace=True)
        return df
    except:
        print('Error loading data')
        return None

##### Downloading the daily stock prices of DJI tickers

In [ ]:
# get DJI components (ticker symbols)
dji_df = fetch_info()
dji_df.head()

/var/folders/nr/kmlljdx179d88t_l2f30j48h0000gr/T/ipykernel_30148/2498928026.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables))[1]


,Company,Exchange,Symbol,Industry,Date added,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,1.54%
1,American Express,NYSE,AXP,Financial services,1982-08-30,3.64%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,4.80%
3,Amazon,NASDAQ,AMZN,Retailing,2024-02-26,2.93%
4,Apple,NASDAQ,AAPL,Information technology,2015-03-19,3.04%


In [ ]:
tickers = dji_df.Symbol.values.tolist()

In [ ]:
start_date = "2021-01-01"
end_date = "2022-09-01"
df = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  30 of 30 completed


In [ ]:
# use the adjusted closing prices for follow-up analysis
df = df['Adj Close']

In [ ]:
df

Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,126.830078,202.947906,159.331497,112.457108,202.720001,169.422394,219.680618,39.525219,73.209152,176.289490,...,123.086594,69.399284,211.224304,134.810303,126.315659,125.758568,332.521729,212.323700,47.345634,46.305019
2021-01-05,128.398178,203.932816,160.925507,113.057312,211.630005,170.808289,220.887177,39.543198,75.188240,177.043564,...,122.878891,69.519318,211.428085,135.897644,127.122192,124.720245,328.049957,209.154831,47.128414,46.058533
2021-01-06,124.076096,208.848465,156.919006,117.239685,211.029999,180.314178,215.532516,39.920837,77.608093,177.718216,...,124.748085,70.616531,205.945877,136.975372,128.460297,128.957062,341.817047,207.312027,47.699619,46.346100
2021-01-07,128.309982,209.537918,158.108002,116.312325,212.710007,180.658325,217.357300,40.424339,77.971069,177.182449,...,121.546837,72.022346,211.806488,139.477173,127.259666,128.929184,347.297241,208.472321,47.592289,46.342941
2021-01-08,129.417450,213.540314,159.134995,116.427025,209.899994,180.686249,221.405685,40.514248,78.714302,177.291595,...,119.326759,71.173706,213.097000,140.824356,127.204651,129.466919,345.736908,210.071335,46.998699,46.336617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-25,168.361389,232.346207,137.279999,157.784500,169.380005,192.002197,173.413177,44.345840,153.929611,116.540764,...,108.684410,85.480461,274.618378,110.287445,139.306488,161.899994,525.921021,206.933151,38.154915,44.138420
2022-08-26,162.014313,227.589249,130.750000,152.999374,164.529999,185.282898,164.757965,43.051212,152.798187,113.177299,...,98.315384,84.524063,264.021667,105.475395,136.046112,159.849640,513.956909,200.098480,37.727486,42.751278
2022-08-29,159.796280,226.388153,129.789993,150.305237,165.419998,185.147736,159.752335,42.882347,153.938980,112.641525,...,96.259850,82.942673,261.205078,105.085762,135.472443,158.431671,514.461975,199.546188,37.797268,43.167084


##### Generating monthly returns from daily prices

In [ ]:
mth_return_df = df.pct_change().resample("M").agg(lambda x: (x+1).prod()-1)

/var/folders/nr/kmlljdx179d88t_l2f30j48h0000gr/T/ipykernel_30148/2780784172.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mth_return_df = df.pct_change().resample("M").agg(lambda x: (x+1).prod()-1)


In [ ]:
mth_return_df


Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,0.019705,0.065164,0.006141,-0.011626,-0.042078,0.009129,0.023830,0.014104,0.005784,-0.053523,...,0.022051,-0.048048,0.065552,-0.046467,-0.064118,0.004792,-0.045551,-0.112555,-0.059709,-0.041220
2021-02-28,-0.079712,-0.061457,-0.035328,0.163427,0.091766,0.180704,-0.040167,0.006505,0.190300,0.124101,...,0.004797,-0.057740,0.004118,0.010959,-0.036503,0.067498,-0.004077,0.100749,0.010046,-0.075237
2021-03-31,0.007340,0.106216,0.000372,0.048833,0.201453,0.074069,-0.021386,0.152441,0.047900,-0.023910,...,0.100651,0.070880,0.014588,-0.014023,0.096333,0.039349,0.123930,-0.003108,0.051537,0.049855
2021-04-30,0.076218,-0.036855,0.120663,0.084205,-0.080127,-0.011769,0.087082,-0.008413,-0.016414,0.008129,...,0.023147,-0.033597,0.069602,-0.002032,-0.008539,0.028325,0.071841,0.103103,0.004501,0.030037
2021-05-31,-0.050497,-0.000113,-0.070470,0.044212,0.054244,0.056858,0.033779,0.039088,0.019312,-0.039619,...,0.037507,0.018658,-0.007627,0.031031,0.010718,0.032587,0.032899,-0.025389,-0.022495,0.019139
2021-06-30,0.099109,0.024418,0.067355,0.031849,-0.030200,-0.097271,0.025913,0.001891,0.009153,-0.016121,...,-0.021720,0.083193,0.084989,0.132127,0.000593,-0.057289,-0.024309,0.028685,-0.008143,-0.007111
2021-07-31,0.064982,-0.009067,-0.032722,0.034770,-0.054600,-0.044895,-0.009580,0.052001,-0.027974,0.001422,...,-0.003474,-0.011573,0.051716,0.084277,0.060732,-0.005277,0.029418,0.053759,0.006722,0.010849
2021-08-31,0.042489,-0.059073,0.043034,-0.026799,-0.030819,0.019928,0.096474,0.065920,-0.036689,0.029997,...,-0.008657,-0.007545,0.061591,-0.014914,0.001125,0.072455,0.009825,-0.068906,-0.013983,0.042757
2021-09-30,-0.068037,-0.057110,-0.053518,0.009460,0.002005,-0.089629,0.022430,-0.077770,0.048362,-0.066906,...,-0.099209,-0.006644,-0.066119,-0.118429,-0.018189,-0.042907,-0.058036,-0.027717,-0.018000,-0.058879


In [ ]:
# creating a series with 9 one minute timestamps
index = pd.date_range('1/1/2000', periods=9, freq='T')
series = pd.Series(range(9), index=index)
series

/var/folders/nr/kmlljdx179d88t_l2f30j48h0000gr/T/ipykernel_30148/1448798986.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  index = pd.date_range('1/1/2000', periods=9, freq='T')


2000-01-01 00:00:00    0
2000-01-01 00:01:00    1
2000-01-01 00:02:00    2
2000-01-01 00:03:00    3
2000-01-01 00:04:00    4
2000-01-01 00:05:00    5
2000-01-01 00:06:00    6
2000-01-01 00:07:00    7
2000-01-01 00:08:00    8
Freq: min, dtype: int64

In [ ]:
series.resample('3T').sum()

/var/folders/nr/kmlljdx179d88t_l2f30j48h0000gr/T/ipykernel_30148/3410253585.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  series.resample('3T').sum()


2000-01-01 00:00:00     3
2000-01-01 00:03:00    12
2000-01-01 00:06:00    21
Freq: 3min, dtype: int64

##### Calculating six-month cumulative returns

In [ ]:
# obtain the historical cumulative returns of past 6 months as the terminal return of current month
past_cum_return_df = (mth_return_df+1).rolling(6).apply(np.prod) - 1

In [ ]:
past_cum_return_df

Ticker,AAPL,AMGN,AMZN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,...,MMM,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-06-30,0.061724,0.091013,0.079561,0.408910,0.181728,0.206571,0.108756,0.214285,0.269281,-0.010750,...,0.173797,0.024281,0.250176,0.107184,-0.008763,0.115939,0.154033,0.076980,-0.027373,-0.029790
2021-07-31,0.108867,0.014981,0.037861,0.475046,0.166280,0.141977,0.072575,0.259663,0.226678,0.046679,...,0.144482,0.063528,0.233944,0.258996,0.123471,0.104757,0.244678,0.278815,0.041342,0.022901
2021-08-31,0.256109,0.017560,0.122169,0.233869,0.035329,-0.013526,0.225266,0.334022,-0.007248,-0.040944,...,0.129159,0.120183,0.304571,0.226776,0.167346,0.109886,0.262052,0.081715,0.016569,0.153416
2021-09-30,0.162118,-0.132676,0.061718,0.187550,-0.136542,-0.163874,0.280125,0.067539,-0.006810,-0.083189,...,-0.075878,0.039089,0.200797,0.096871,0.045406,0.022048,0.057725,0.055012,-0.050656,0.033956


### Generating Trading Signals

In [ ]:
import datetime as dt
end_of_measurement_period = dt.datetime(2022,6,30)
formation_period = dt.datetime(2022,7,31)

In [ ]:
end_of_measurement_period_return_df =past_cum_return_df.loc[end_of_measurement_period]
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
end_of_measurement_period_return_df.head()

,Ticker,2022-06-30 00:00:00
0,AAPL,-0.227937
1,AMGN,0.099514
2,AMZN,-0.362932
3,AXP,-0.144964
4,BA,-0.320882


In [ ]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Ticker                      CVX
2022-06-30 00:00:00    0.256955
Name: 8, dtype: object

In [ ]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Ticker                      DIS
2022-06-30 00:00:00   -0.390535
Name: 9, dtype: object

In [ ]:
pd.qcut(series, 5, labels=False)

2000-01-01 00:00:00    0
2000-01-01 00:01:00    0
2000-01-01 00:02:00    1
2000-01-01 00:03:00    1
2000-01-01 00:04:00    2
2000-01-01 00:05:00    3
2000-01-01 00:06:00    3
2000-01-01 00:07:00    4
2000-01-01 00:08:00    4
Freq: min, dtype: int64

##### Rank-ordering the stocks based on cumulative terminal monthly returns

In [ ]:
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 5, labels=False)
end_of_measurement_period_return_df.head()

,Ticker,2022-06-30 00:00:00,rank
0,AAPL,-0.227937,1
1,AMGN,0.099514,4
2,AMZN,-0.362932,0
3,AXP,-0.144964,2
4,BA,-0.320882,0


##### Obtaining the stock tickers to long or short

In [ ]:
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==4,"Ticker"].values
long_stocks

array(['AMGN', 'CVX', 'IBM', 'KO', 'MRK', 'TRV'], dtype=object)

In [ ]:
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Ticker"].values
short_stocks

array(['AMZN', 'BA', 'CRM', 'DIS', 'HD', 'NKE'], dtype=object)

##### Obtaining the performance of stocks in a long position at the evaluation period

In [ ]:
long_return_df = mth_return_df.loc[formation_period + relativedelta(months=1), mth_return_df.columns.isin(long_stocks)]
long_return_df

Ticker
AMGN   -0.021474
CVX    -0.026156
IBM    -0.005517
KO     -0.038336
MRK    -0.044549
TRV     0.018526
Name: 2022-08-31 00:00:00, dtype: float64

##### Obtaining the performance of stocks in a short position at the evaluation period

In [ ]:
short_return_df = mth_return_df.loc[formation_period + relativedelta(months=1),  mth_return_df.columns.isin(short_stocks)]
short_return_df

Ticker
AMZN   -0.060615
BA      0.005900
CRM    -0.151614
DIS     0.056362
HD     -0.035350
NKE    -0.073703
Name: 2022-08-31 00:00:00, dtype: float64

In [ ]:
momentum_profit = long_return_df.mean() - short_return_df.mean()
momentum_profit


0.023585513013341462

### Comparing with the Buy-and-Hold Strategy

In [ ]:
df_dji = yf.download("^DJI", start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


##### Calculating the monthly terminal returns of the buy-and-hold strategy

In [ ]:
buy_n_hold_df = df_dji['Adj Close'].pct_change().resample("M").agg(lambda x: (x+1).prod()-1)
buy_n_hold_df.head()

/var/folders/nr/kmlljdx179d88t_l2f30j48h0000gr/T/ipykernel_30148/3477312173.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  buy_n_hold_df = df_dji['Adj Close'].pct_change().resample("M").agg(lambda x: (x+1).prod()-1)


Date
2021-01-31   -0.007983
2021-02-28    0.031677
2021-03-31    0.066247
2021-04-30    0.027085
2021-05-31    0.019324
Freq: ME, Name: Adj Close, dtype: float64

In [ ]:
buy_n_hold_df.loc[formation_period + relativedelta(months=1),]


-0.04063613884907047

########################################################################################################################